In [ ]:
import copy
import json
import math
import random

from typing import Any

In [ ]:
class Model:
    def __init__(
        self,
        input_size: list[list[int]],
        padding_size: int,
        weight: list[list[list[float]]],
        conv_filter_count: int,
        conv_filter_size: list[tuple[int]],
        conv_stride_size: list[int],
        pool_filter_size: tuple[int],
        pool_stride_size: int,
        pool_mode: str,
        dense_unit_count: int,
        dense_activation: str,
    ) -> None:
        self._input_size = input_size
        self._padding_size = padding_size
        self._weight = weight
        self._conv_filter_count = conv_filter_count
        self._conv_filter_size = conv_filter_size
        self._conv_stride_size = conv_stride_size
        self._pool_filter_size = pool_filter_size
        self._pool_stride_size = pool_stride_size
        self._pool_mode = pool_mode
        self._dense_unit_count = dense_unit_count
        self._dense_activation = dense_activation
        self._convolution_layer = self.ConvolutionLayer()
        self._detector_layer = self.DetectorLayer()
        self._pooling_layer = self.PoolingLayer()
        self._dense_layer = self.DenseLayer()
        self._flatten_layer = self.FlattenLayer()

    class ConvolutionLayer:
        def __init__(self) -> None:
            pass

        def convolute(self) -> None:
            pass

    class DetectorLayer:
        def __init__(self) -> None:
            pass

        def detect(self) -> None:
            pass

    class PoolingLayer:
        def __init__(self) -> None:
            pass

        def pool(self) -> None:
            pass

    class DenseLayer:
        def __init__(self) -> None:
            pass

        def dense(self) -> None:
            pass

    class FlattenLayer:
        def __init__(self) -> None:
            pass

        def flatten(self) -> None:
            pass

    def feedforward(self) -> None:
        self._convolution_layer.convolute()
        self._detector_layer.detect()
        self._pooling_layer.pool()
        self._dense_layer.dense()
        self._flatten_layer.flatten()

    def back_propagate(self) -> None:
        pass